In [1]:
import sys
import os
# This code enables using of "src.data" imports in vs code (when you're launching it directly from notebooks directory)
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(project_root)

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import transformers

torch.manual_seed(42)

In [2]:
# Loading model weights
qconf = transformers.BitsAndBytesConfig(load_in_8bit=True)

model_name = "AnatoliiPotapov/T-lite-instruct-0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="cuda:0",
    torch_dtype="auto",
    quantization_config=qconf,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
from src.data.classification import SST2Dataset

test_sst2_ds = SST2Dataset(
    tokenizer=tokenizer,
    data_path="../../data/sst-2/test-00000-of-00001.parquet",
    config_path="../../data/",
    device=model.device
)
test_sst2_ds.prompt

'Please perform Sentiment Classification task\n\nAnswer using the label from [negative, positive].\nGenerate the final answer bracketed with <ans> and </ans>.\n\nThe input:\n<INPUT>\n\nResponse:\n'

In [5]:
from src.evaluation.evaluator import TextClassificationEvaluator

# terminators were taken from hf model page (t-lite 0.1)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

model_generate_params = {
    "max_new_tokens": 50,
    "eos_token_id": terminators
}

evaluator = TextClassificationEvaluator()
macro_f1 = evaluator.evaluate(
    model=model, 
    tokenizer=tokenizer,
    eval_ds=test_sst2_ds,
    batch_size=64,
    model_generate_args = model_generate_params
)
macro_f1

  0%|          | 0/29 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/nfs/home/edyagin/.virtualenvs/prompt_optimization/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 29/29 [01:14<00:00,  2.57s/it]


0.5565279674105245

In [6]:
torch.cuda.empty_cache()

# Generation

In [7]:
from src.data.generation import SamsumDataset

gen_ds = SamsumDataset(
    tokenizer=tokenizer,
    data_path="../../data/samsum/test-00000-of-00001.parquet",
    config_path='../../data',
    device=model.device
)
gen_ds.prompt

'INSTRUCTION:\nSummarize the following text\n\nINPUT:\n<INPUT>\n\nRESPONSE:\n'

In [8]:
from src.evaluation.evaluator import GenerationEvaluator

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

model_generate_params = {
    "max_new_tokens": 256,
    "eos_token_id": terminators
}

evaluator = GenerationEvaluator()
metrics = evaluator.evaluate(
    model=model, 
    tokenizer=tokenizer,
    eval_ds=gen_ds,
    batch_size=32,
    model_generate_args = model_generate_params
)
metrics

[nltk_data] Downloading package wordnet to
[nltk_data]     /nfs/home/edyagin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /nfs/home/edyagin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /nfs/home/edyagin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
  0%|          | 0/26 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/nfs/home/edyagin/.virtualenvs/prompt_optimization/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 26/26 [18:19<00:00, 42.29s/it]


{'bleu': 0.0983179127962664,
 'rouge': 0.3313492109838392,
 'meteor': 0.4484997598087172}